In [1]:
import os
import time
import numpy as np
import tensorflow as tf

In [2]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

In [7]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [8]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [9]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

In [10]:
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [13]:
seq_length = 100

In [14]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [15]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print(f'Input: {text_from_ids(input_example).numpy()}')
    print(f'Target: {text_from_ids(target_example).numpy()}')

Input: b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [20]:
BATCH_SIZE = 64

# Buffer size to shuffle dataset (TF data is designed to work with possibly infinite sequences, so it doesn't attemp to shuffle the entire sequence in memory. Instead, it maintains a buffer in wich it shuffles elements)
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup layer
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )

        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)

        if states is None:
            states = self.gru.get_initial_state(x)

        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states

        else:
            return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(f'{example_batch_predictions.shape} # (batch_size, sequence_length, vocab_size)')

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([23, 56, 53, 16, 38, 40, 42, 27, 33, 23, 30, 51, 36, 10, 20, 61, 22,
       23, 16, 21, 52, 55, 52, 14, 62, 46, 28, 53, 47, 57, 25, 32, 11,  8,
       27, 13, 26, 53, 59,  8, 50,  5, 23, 40,  7, 10, 50, 60,  3, 16,  0,
       60,  9, 62, 58, 13, 53, 40, 47, 59, 12,  2, 46, 62, 47, 18, 51, 42,
       46, 29,  2,  2, 52,  6, 60, 56, 13, 10, 19, 56, 14, 47, 54, 53, 22,
        5, 46, 13, 30,  5, 52, 51, 45,  8, 59, 10, 53, 53, 29, 32],
      dtype=int64)

In [28]:
print(f'Input:\n{text_from_ids(input_example_batch[0]).numpy()}')
print(f'Next char predictions:\n{text_from_ids(sampled_indices).numpy()}')

Input:
b"efore, and yet you fled.\n\nWARWICK:\n'Twas not your valour, Clifford, drove me thence.\n\nNORTHUMBERLAND"
Next char predictions:
b"JqnCYacNTJQlW3GvIJCHmpmAwgOnhrLS:-N?Mnt-k&Ja,3ku!C[UNK]u.ws?naht; gwhElcgP  m'uq?3FqAhonI&g?Q&mlf-t3nnPS"


In [29]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(f'Prediction shape: {example_batch_predictions.shape} # (batch_size, sequence_length, vocab_size)')
print(f'Mean loss: {example_batch_mean_loss}')

Prediction shape: (64, 100, 66) # (batch_size, sequence_length, vocab_size)
Mean loss: 4.191237926483154


In [31]:
tf.exp(example_batch_mean_loss).numpy()

66.10457

In [32]:
model.compile(optimizer='adam', loss=loss)

In [33]:
checkpoint_dir = 'ckpts/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [34]:
EPOCHS = 20

In [35]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/20
172/172 [==============================] - 10s 46ms/step - loss: 2.7329
Epoch 2/20
172/172 [==============================] - 9s 47ms/step - loss: 1.9934
Epoch 3/20
172/172 [==============================] - 10s 56ms/step - loss: 1.7221
Epoch 4/20
172/172 [==============================] - 10s 56ms/step - loss: 1.5616
Epoch 5/20
172/172 [==============================] - 10s 56ms/step - loss: 1.4618
Epoch 6/20
172/172 [==============================] - 10s 57ms/step - loss: 1.3927
Epoch 7/20
172/172 [==============================] - 10s 57ms/step - loss: 1.3409
Epoch 8/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2961
Epoch 9/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2556
Epoch 10/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2157
Epoch 11/20
172/172 [==============================] - 10s 57ms/step - loss: 1.1768
Epoch 12/20
172/172 [==============================] - 10s 57ms/step - loss: 1.1366
Ep

In [39]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generate
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]

        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,

            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )

        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model. `predicted_logits.shape` is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids,
            states=states,
            return_state=True
        )

        # Only use the last prediction
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature

        # Apply the prediction mask: prevent "[UNK]" from being generated
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state
        return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
start = time.time()
states = None

next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

end = time.time()

print(f'{result[0].numpy().decode("utf-8")}\n{"_" * 80}')
print(f'Run time: {end - start}')

ROMEO:
Thou hast I loved the moon. Now too much blood to bear
A bloody skeet o' the loss, is chance to feed,
As to same hither.

DUKE VINCENTIO:
O, work it will not stoop: hold your means makes the base:
The odds for her way raise on, and love no night.

ROMEO:
Fear not, but that's my brother?

ISABELLA:
I'll soon made a consuler,
This we have denited their festless woman?
Here rob his lips and all extremest sheep
To but mercy to take and beast.

LUCENTIO:
Perrain; marry, call the Volscian neighbours. Grey, come you: sir.

SICINIUS:
First, confident. What call's resolve?

BRAKENBURY:
This is stone joints, honesty only safest hit,
Which shows me or so: could feel the soul of him.

EDWARD:
I'll bring me: one, the prove means that your most grave brave king,
My words suffer my cheeks with skins of brains;
Or shall we most morblit: there is many tears together
For comfort, for would be a sad,
to interpett at thine one and gilt thirst may spread of tears,
Conditions and man of Pastival warm

In [42]:
start = time.time()
states = None

next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

end = time.time()

print(f'{result}\n{"_" * 80}')
print(f'Run time: {end - start}')

[b"ROMEO:\nCall me of virtuous sea:\nI from she such ispect light, is it not so?\n\nGREMIO:\nA visil nosely in our think whereof he hath\nMakerved this damn'd for her hand, shake thee my queen.\n\nLord:\nTalten, makes the bow; perhaps with me?\n\nDUKE OF YORK:\nThere's my grim and not meddle words, let it fly.\nUndetch'd on other so.\n\nMARCIUS:\nThey hate us all too soon two every; and\nTherefore they say, with maids, lived. Do not make her weak.\nIf ever you have left you out.\n\nVINCENTIO:\nI know him now; this is so heart: but me\nwith comfort. First, hie you to well; and go about the city;\nIf this induction of the bring thou thou had,\nshall, for our grmanished: if it be,\nAnd those fines of this body stood done!\nWe are not safe; to bear him chosen livery.\n\nThird Servingman:\nWhenceford it is to intend more\nThan to another dukes, commands. If I command,\nAs well appeared in my truth in the book; if phessments death\nIs made an embracherous shame: be still combath\nThe laughte

In [43]:
tf.saved_model.save(one_step_model, 'models')
one_step_reloaded = tf.saved_model.load('models')

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


In [44]:
states = None

next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode('utf-8'))

ROMEO:
I do not like the dew-crowned bagm.
Have we have borne escaped, who lack'st their hearts may
have y


In [45]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs

        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)

        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {'loss': loss}

In [46]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [47]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [48]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 11s 45ms/step - loss: 2.7200


In [49]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])

        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f'Epoch: {epoch + 1} | Batch: {batch_n} | Loss: {logs["loss"]:.4f}'
            print(template)

    # Saving the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print(f'Epoch: {epoch + 1} | Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch: {time.time() - start:.2f} sec')

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch: 1 | Batch: 0 | Loss: 2.1775
Epoch: 1 | Batch: 50 | Loss: 2.0352
Epoch: 1 | Batch: 100 | Loss: 1.9447
Epoch: 1 | Batch: 150 | Loss: 1.8706
Epoch: 1 | Loss: 1.9896
Time taken for 1 epoch: 9.41 sec
Epoch: 2 | Batch: 0 | Loss: 1.8308
Epoch: 2 | Batch: 50 | Loss: 1.7703
Epoch: 2 | Batch: 100 | Loss: 1.6983
Epoch: 2 | Batch: 150 | Loss: 1.6479
Epoch: 2 | Loss: 1.7174
Time taken for 1 epoch: 8.56 sec
Epoch: 3 | Batch: 0 | Loss: 1.6298
Epoch: 3 | Batch: 50 | Loss: 1.5776
Epoch: 3 | Batch: 100 | Loss: 1.5237
Epoch: 3 | Batch: 150 | Loss: 1.4988
Epoch: 3 | Loss: 1.5571
Time taken for 1 epoch: 9.92 sec
Epoch: 4 | Batch: 0 | Loss: 1.4679
Epoch: 4 | Batch: 50 | Loss: 1.4857
Epoch: 4 | Batch: 100 | Loss: 1.4549
Epoch: 4 | Batch: 150 | Loss: 1.4414
Epoch: 4 | Loss: 1.4582
Time taken for 1 epoch: 9.94 sec
Epoch: 5 | Batch: 0 | Loss: 1.4039
Epoch: 5 | Batch: 50 | Loss: 1.3971
Epoch: 5 | Batch: 100 | Loss: 1.3490
Epoch: 5 | Batch: 150 | Loss: 1.3835
Epoch: 5 | Loss: 1.3895
Time taken for 1 epoch: